## **Aim:- Blur the specific  object present in the video**
### **Author: Shalu Singh**
### **Date: 12/09/23**


In [1]:
# # mount google drive
# from google.colab import drive
# # dr jnbive.mount('/content/drive/')

In [2]:
# import libraries
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow_hub as hub

In [3]:
# function: blur the image
def blur_image(image,coordinates = None):
  img = image.copy() # copy the image to work on new image
  if (coordinates is not None):
    #print('Performing image blur operation...')
    for coord in (coordinates):
      ymin,xmin,ymax,xmax = coord
      #print('Image shape:',img.shape)
      # Extract region of intrest
      Y_min,X_min,Y_max,X_max = int(ymin*img.shape[0]),int(xmin*img.shape[1]),int(ymax*img.shape[0]),int(xmax*img.shape[1])
      #print('Y_min,Y_max',Y_min,Y_max)
      #print('X_min,X_max',X_min,X_max)
      roi = img[Y_min:Y_max,X_min:X_max]
      #show_img(roi,'Original_roi')
      # blur the extracted img using Gausian blur
      try:
        roi = cv2.GaussianBlur(roi,(7,7),5)
        #show_img(roi,title='blured roi')
        # replace the original roi with blured_roi
        img[Y_min:Y_max, X_min:X_max] = roi

      except:
        pass

    return img


In [4]:
# function: get image info.
def get_info(img):
  print('Image shape:',img.shape)
  print('Image Dtype:',img.dtype)
  print('Image Min. Value:',tf.reduce_min(img))
  print('Image Max. Value:',tf.reduce_max(img))

In [5]:
# function: to show the image
def show_img(img,title = ''):
  try:
    img = img.astype('uint8')
    plt.imshow(img)
    plt.axis('off')
    plt.title(title)
    plt.show()
  except:
    print('IMAGE NOT SHOWN!')

In [6]:
# detector_boxes = detector_output['detection_boxes']
# det_score = detector_output['detection_scores']
# sel_boxes = detector_boxes[det_score > 0.5].numpy()
# ymin,xmin,ymax,xmax = sel_boxes[0]
# print(ymin,xmin,ymax,xmax)
# ymin,xmin,ymax,xmax = int(ymin*original_img.shape[1]),int(xmin*original_img.shape[2]),int(ymax*original_img.shape[1]),int(xmax*original_img.shape[2])
# print(ymin,xmin,ymax,xmax)

In [7]:
# k = original_img[0]
# k = k[ymin:ymax,xmin:xmax]
# show_img(k)

In [8]:
# function: filter detection boxs
def filter_detection(detector_output,thr = 0.5):
  detection_boxs = detector_output['detection_boxes']
  detection_class = detector_output['detection_classes']
  detection_scores = detector_output['detection_scores']

  # filter the detection boxses based on threshold
  selected_scores = detection_scores[detection_scores >= thr]
  selected_class = detection_class[detection_scores >= thr]
  selected_boxs = detection_boxs[detection_scores >= thr].numpy()

  return selected_boxs,selected_class,selected_scores

In [9]:
# function: to get info. of detection output
def detection_info(detector_output):
  print('Number of detection:',detector_output['num_detections'][0].numpy())
  print("Detection class label:",detector_output['detection_classes'][0].numpy())
  #print('Detection boxes coordinates:',detector_output['detection_boxes'])
  print('Detection scores:',detector_output['detection_scores'][0].numpy())


In [10]:
# # function: draw bounding box in an image
# def bounding_box(img,coordinates):
#   # define the color of the bouding box
#   color = (0,255,0) # (BGR)
#     # box thickness
#   thickness = 2
#   #  convert coordinates to integer
#   for i in coordinates:
#     y_min,x_min,y_max,x_max = i.astype('uint8')

#     # Drawing the bounding box on the image
#     img = cv2.rectangle(img,(x_min,y_min),(x_max,y_max),color,thickness)
#   # show the image
#   print('Bounding box image')
#   show_img(img)



In [11]:
# # function to load the form local disk
# def load_img(img_path):
#   # check if image path exist or not
#   if not(os.path.exists(img_path)):
#     print("Img not found!")
#     return None
#   else:
#     print('Img found')
#     img = Image.open(img_path)
#     print('Image loaded successfully!')
#     # convert the image to tensor
#     img = tf.convert_to_tensor(img)
#     # resize the image to shape: 520,520,chanel
#     img = tf.image.resize(img,(520,520))
#     # get img info
#     get_info(img)
#     # add additional dimension to image
#     # new shape: batch,height,width,chanel
#     img = tf.expand_dims(img,axis = 0) # add unit batch size
#     # convert to numpy
#     # get new img info
#     get_info(img)
#     img = np.asarray(img)
#     return img
#     # except:
#     #   print('Failed to load image')
#     #   return None


In [12]:
# # testing the load img function
# img_path = '/content/drive/MyDrive/Dataset/ped_corss.jpg'
# original_img = load_img(img_path)

In [13]:
# show_img(original_img[0],'Original Image')

##### Load the object detection model: **ssd_mobilenet_v2**


In [26]:
# #model_path =  'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2'
object_detection_model = tf.saved_model.load('mobilenet_model/')

In [27]:
# # test loaded model
# detector_output = object_detection_model(original_img)

In [28]:
# # get filtered detection output
# boxes,classes,scores = filter_detection(detector_output)

In [29]:
# # blur the image
# output_img = blur_image(original_img,boxes)

In [30]:
# get_info(output_img)

In [31]:
# # resulted image
# show_img(output_img,title = 'Output Image')

### Image capturing and bluring image

In [32]:
# import opencv
import cv2

In [33]:
# load video from local disk
ip_video_path = 'Hit_Video.mp4'
# laod the video
try:
  cap = cv2.VideoCapture(ip_video_path)
  print('Video loaded successfully!')
except:
  print("Failed! to load video")

Video loaded successfully!


In [34]:
# get video property like frame_width,frame_heigh,frame_per_second(fps),codecc
frame_width = int(cap.get(3)) # width of the fames in the video
frame_height = int(cap.get(4)) # height of the frame in the video
fps = int(cap.get(5)) # frame per second
codecc = cv2.VideoWriter_fourcc(*'XVID') # codecc for output video (XVID default codecc)

In [35]:
print('Frame Width:',frame_width)
print('Frame height:',frame_height)
print('Frame Per Second:',fps)

Frame Width: 640
Frame height: 360
Frame Per Second: 30


In [36]:
# VideoWriter object to save blured video
op_video_path = 'output_blured_video.mp4'
out = cv2.VideoWriter(op_video_path,codecc,fps,(frame_width,frame_height))

In [37]:
# Loop through each video frame,blured the image and write in output video
while True:
  ret, frame = cap.read() # Read frame from the input image
  #print('Original frame shape:',frame.shape)
  #print('fame shape:',frame.shape)
  # Expand frame dimension
  if ret:
    frame = tf.expand_dims(frame,axis = 0)
  else:
    break # break if we have reached the end of the video
  # Blur the image
  detector_output = object_detection_model(frame)
  boxes,classes,scores = filter_detection(detector_output)

  #print('New frame shape:',frame[0].shape)
  blured_img = blur_image(frame[0].numpy(),boxes)
  #print('blured_img_shape',blured_img.shape)
  # write blured frame to output image
  out.write(blured_img)
  # Display original and blured frame
  #show_img(frame[0].numpy(),'Original_frame')
  #show_img(blured_img,'Blured_frame')


  #break the loop if key 'q' is pressed
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break


# realease video capture and writer object
cap.release()
out.release()

# close all opencv windows
cv2.destroyAllWindows()


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [ ]:
  for i in n_coordin:
    y_min,x_min,y_max,x_max = i.astype('uint8')